# Detecting Vehicles

Here we are trying to create a model to detect 10 types of vehicles.

This is a dataset Arunoda created based on Google search images.
You can get it from [here](https://github.com/arunoda/fastai-courses/releases/tag/fastai-vehicles-dataset).

## Setting Up
This is how we install fastai and setup our notebook

In [0]:
!pip install fastai2
!git clone https://github.com/arunoda/deeplearning.git
%cd 'deeplearning'

In [0]:
from fastai2.vision.all import *

## Getting Data

Here we will download a dataset hosted on GitHub.<br/>
Unfortunately, `untar_data` does not work with here. So, use the following `download.sh` bash script.

In [0]:
dataset_url = "https://github.com/arunoda/fastai-courses/releases/download/fastai-vehicles-dataset/fastai-vehicles.tgz"

In [0]:
!./download.sh {dataset_url}

In [0]:
im_path = Path("/content/deeplearning/.datasets/77c6f8768e1dcbe1935d260cedfad9329e025c93")
im_path.ls()

In [0]:
image_list = (im_path/"images").ls()
image_list

In [0]:
print
show_image(Image.open(image_list[0]))

## Let's load Images

In [0]:
def get_vehicle_type(p):
    return re.sub('-[0-9]+.*', '', p.name)

get_vehicle_type(image_list[0])

In [0]:
db = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(0.3),
    get_y=get_vehicle_type,
    item_tfms=Resize(224)
)

In [0]:
dls = db.dataloaders(im_path/"images")

In [0]:
dls.show_batch()

In [0]:
## These are the types of vehicles we have.
dls.vocab

## Let's Create Our Model

In [0]:
learn = cnn_learner(dls, resnet34, metrics=accuracy)

In [0]:
learn.fine_tune(4)

## Let's Analyze Our Model

In [0]:
analyzer = ClassificationInterpretation.from_learner(learn)

In [0]:
analyzer.plot_confusion_matrix(figsize=(8,8))

In [0]:
analyzer.plot_top_losses(9, figsize=(12, 12))

## Cleaning Data

In [0]:
from fastai2.vision.widgets import *

In [0]:
cleaner = ImageClassifierCleaner(learn)

In [0]:
cleaner

In [0]:
for idx in cleaner.delete():
    print(cleaner.fns[idx])
    cleaner.fns[idx].unlink()

## Run the model again

In [0]:
dls = db.dataloaders(im_path)

In [0]:
learn = cnn_learner(dls, resnet34, metrics=accuracy)

In [0]:
learn.fine_tune(4)